In [11]:
import requests
import urllib
import time
import csv
import tqdm

In [2]:
def runQuery(query, start_year=1850,
			end_year=1860, corpus=26,
			smoothing=0):

    # converting a regular string to
    # the standard URL format
    # eg: "geeks for,geeks" will
    # convert to "geeks%20for%2Cgeeks"
    query = urllib.parse.quote(query)
    # print(query)
    # creating the URL
    url = 'https://books.google.com/ngrams/json?content=' + query +'&year_start=' + str(start_year) + '&year_end=' +str(end_year) + '&corpus=' + str(corpus) + '&smoothing=' +str(smoothing) + ''
    #print(url)
    time.sleep(1)
    
    # requesting data from the above url
    response = requests.get(url)
    
    #print(response.status_code)
    #if response.status_code != 200:
    #    return(query)
    #     for i in range(10):
    #         response = requests.get(url)
    #         if response.status_code == 200:
    #             continue
    # extracting the json data from the response we got
    # print(response.status_code)
    # print(type(response))
    
    output = response.json()
    
    # creating a list to store the ngram data
    return_data = []
    
    if len(output) == 0:
        # if no data returned from site,
        # print the following statement
        return "No data available for this Ngram."
    else:
        # if data returned from site,
        # store the data in return_data list
        for num in range(len(output)):
            
            # getting the name
            return_data.append((output[num]['ngram'],
                                
                                # getting ngram data
                                output[num]['timeseries'])
                            )
    
    return return_data


In [3]:
#query = "Albert Einstein"
#print(runQuery(query))
#print(runQuery(query,start_year=1900, end_year=2000))
#my_queries = list()
#my_queries = ["brain", "cognitive"]
#my_result = {}
#for query in my_queries:
#    my_result[query] = runQuery(query,start_year=1900, end_year=2000)
# my_result["brain"]

In [4]:
def txtfile_to_list(path):
    #read a stimulus text file into a list
    
    lines_as_lists = []
    
    # Open the text file for reading
    with open(path, 'r') as file:
        # Loop through each line in the file
        for line in file:
            # Split the line into words using commas as the delimiter
            words = line.strip().split(',')
            
            # Join the words with spaces instead of commas
            line_without_commas = ' '.join(words)
            
            # Append the modified line to the list
            lines_as_lists.append(line_without_commas)
    
    # Now, 'lines_as_lists' contains each line from the file as a list of words with spaces instead of commas
    return lines_as_lists

In [18]:
def calc_ngrams_from_list(list, startyear, endyear):
    my_result = []
    ii = 0
    for query in tqdm.tqdm(list[ii:]):
        result = runQuery(query,startyear, endyear)
        my_result.append(result)   
        ii= ii+1
    return my_result

In [16]:
def writeCSV_from_result(csv_file_path, entries, my_results, startyear, endyear):
    # Create a list to store the data rows for the CSV
    data_rows = []

    # Iterate over entries and my_results in parallel
    for entry, result in zip(entries, my_results):
        # Create a dictionary to represent each row
        row_data = {'Entry': entry}

        # Handle the case where there is no data available
        if isinstance(result, str) and result == 'No data available for this Ngram':
            row_data['Text'] = result
            # Initialize data columns with None
            for year in range(startyear, endyear):
                row_data[str(year)] = None
        elif isinstance(result, list) and len(result) == 1 and isinstance(result[0], tuple) and len(result[0]) == 2:
            text, values = result[0]
            row_data['Text'] = text
            # Initialize data columns with values or None if not enough values are provided
            for year, value in zip(range(startyear, endyear), values):
                row_data[str(year)] = value if year - startyear < len(values) else None
        else:
            # For invalid entries, store 'No data available for this Ngram' in Text and None in data columns
            row_data['Text'] = 'No data available for this Ngram'
            for year in range(startyear, endyear):
                row_data[str(year)] = None

        # Append the row data to the list of data rows
        data_rows.append(row_data)

    # Open the CSV file for writing
    with open(csv_file_path, 'w', newline='') as csvfile:
        fieldnames = ['Entry', 'Text'] + [str(year) for year in range(startyear, endyear)]
        csvwriter = csv.DictWriter(csvfile, fieldnames=fieldnames)

        # Write the header row
        csvwriter.writeheader()

        # Write the data rows
        csvwriter.writerows(data_rows)

    print(f'CSV file has been saved to {csv_file_path}')


In [13]:
ngram_dir = 'stimuli/ngram/';
ngram_ns = [number for number in range(1, 5)]
startyear = 2010
endyear = 2020
nyears = endyear - startyear

In [20]:
for n in range(1,2):
    # run for sentences:
    print('SENT ngram #' + str(n))
    sentences_stim_path = ngram_dir + 'stimulus_sentences_ngram_n' + str(n) + '.csv';
    result_csv_file_path = ngram_dir + 'frequency_sentences_ngram_n' + str(n) + '.csv';
    lines_as_list = txtfile_to_list(sentences_stim_path)
    my_result = calc_ngrams_from_list(lines_as_list[1:], startyear, endyear)
    writeCSV_from_result(result_csv_file_path, lines_as_list[1:], my_result, startyear, endyear)

    # run for word lists:
    print('WORD ngram #' + str(n))
    words_stim_path = ngram_dir + 'stimulus_words_ngram_n' + str(n) + '.csv';
    result_csv_file_path = ngram_dir + 'frequency_words_ngram_n' + str(n) + '.csv';
    lines_as_list = txtfile_to_list(words_stim_path)
    my_result = calc_ngrams_from_list(lines_as_list[1:], startyear, endyear)
    writeCSV_from_result(result_csv_file_path, lines_as_list[1:], my_result, startyear, endyear)   

SENT ngram #1


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 640/640 [12:56<00:00,  1.21s/it]


CSV file has been saved to stimuli/ngram/frequency_sentences_ngram_n1.csv
WORD ngram #1


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 640/640 [13:09<00:00,  1.23s/it]

CSV file has been saved to stimuli/ngram/frequency_words_ngram_n1.csv
